In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# filepath = Path(__file__).parent
filepath = Path(os.path.abspath(''))
print(filepath)

# infer_dir = filepath / "out_infer"
models_dir = filepath / 'pred_models'
model_name  = 'DeepTTC'
# model_name  = 'GraphDRP'
# model_name  = 'HIDRA'
# model_name  = 'IGTD'
# model_name  = 'PaccMann_MCA'
infer_dir = models_dir / model_name / "out_infer"

preds_fname = "test_y_data_predicted.csv"

canc_col_name = "improve_sample_id"
drug_col_name = "improve_chem_id"

target = "PDMR"

ov_drug_info = pd.read_csv(filepath / "ovarian_data/raw_data/y_data/Drugs_For_OV_Proposal_Analysis.txt", sep="\t")

/Users/apartin/Library/CloudStorage/Box-Box/AP_work/Proposals/Ovarian/GraphDRP


In [2]:
def agg_pred_files(res_dir: Path):
    dfs = []
    split_dirs = list(res_dir.glob("split_*"))
    for sdir in split_dirs:
        preds_fpath = sdir / preds_fname
        if preds_fpath.exists():
            split = str(sdir).split("split_")[1]
            df = pd.read_csv(preds_fpath)
            df["split"] = int(split)
            dfs.append(df)
    rr = pd.concat(dfs, axis=0)#.sort_values(["split"]).reset_index(drop=True)
    rr = rr.sort_values(["split", canc_col_name, drug_col_name]).reset_index(drop=True)
    return rr


def agg_pred_scores(df: pd.DataFrame):
    group_by_cols = ['improve_sample_id', 'improve_chem_id']
    ff = df.groupby(group_by_cols).agg(
        pred_mean=('auc_pred', 'mean'),
        pred_std=('auc_pred', 'std'))
    ff = ff.reset_index().sort_values(['improve_chem_id', 'improve_sample_id', 'pred_mean']).reset_index(drop=True)
    # ff = ff.reset_index().sort_values(['pred_mean', 'improve_chem_id', 'improve_sample_id']).reset_index(drop=True)
    return ff

In [3]:
# CTRP
source = "CTRPv2"
res_dir = infer_dir / f"{source}-{target}"
cc = agg_pred_files(res_dir)
ca = agg_pred_scores(cc)
# ca = ca.sort_values('pred_mean').reset_index(drop=True)

# GDSC
source = "GDSCv2"
res_dir = infer_dir / f"{source}-{target}"
gg = agg_pred_files(res_dir)
ga = agg_pred_scores(gg)
# ga = ga.sort_values('pred_mean').reset_index(drop=True)

ValueError: No objects to concatenate

In [ ]:
# print(ca.shape)
# ca[:5]

# print(ga.shape)
# ga[:5]

In [ ]:
kk = ov_drug_info[[drug_col_name, 'drug_name']].sort_values(drug_col_name)
# print(kk)
ca = ca.merge(kk, on=drug_col_name, how="inner").drop_duplicates()
ga = ga.merge(kk, on=drug_col_name, how="inner").drop_duplicates()

In [ ]:
# df = ca.copy()
# df_pivot = df.pivot(index='drug_name', columns=canc_col_name, values=['pred_mean', 'pred_std'])
# df_pivot

In [ ]:
# df_pivot.columns
# df_pivot = df_pivot.sort_values(('pred_mean', '655913~031-T'))
# df_pivot

In [ ]:
def gen_plot(df: pd.DataFrame, fpath: str, tr_data: str, errorbar: bool=False, show: bool=False):
    """ ... """

    # # Provided DataFrame
    # data = {
    #     'improve_sample_id': ['655913~031-T'] * 16 + ['937885~149-R'] * 16,
    #     'improve_chem_id': ['Drug_1105', 'Drug_1127', 'Drug_1173', 'Drug_1335', 'Drug_1363', 'Drug_1485', 'Drug_195', 'Drug_24', 'Drug_42', 'Drug_458', 'Drug_60', 'Drug_687', 'Drug_720', 'Drug_742', 'Drug_888', 'Drug_929',
    #                         'Drug_1105', 'Drug_1127', 'Drug_1173', 'Drug_1335', 'Drug_1363', 'Drug_1485', 'Drug_195', 'Drug_24', 'Drug_42', 'Drug_458', 'Drug_60', 'Drug_687', 'Drug_720', 'Drug_742', 'Drug_888', 'Drug_929'],
    #     'pred_mean': [0.51446, 0.51168, 0.88171, 0.88716, 0.58945, 0.76977, 0.60875, 0.88901, 0.95778, 0.78132, 0.60308, 0.96372, 0.93658, 0.82627, 0.90785, 0.81146,
    #                   0.54210, 0.54533, 0.88836, 0.89869, 0.61914, 0.80194, 0.66576, 0.88526, 0.95472, 0.78690, 0.71252, 0.96288, 0.94399, 0.83376, 0.90808, 0.81382],
    #     'pred_std': [0.083778, 0.099944, 0.040608, 0.022674, 0.051460, 0.048421, 0.062584, 0.063184, 0.016233, 0.047974, 0.122535, 0.010266, 0.031863, 0.047361, 0.027914, 0.051955,
    #                  0.112399, 0.116313, 0.037299, 0.024605, 0.049752, 0.040869, 0.070850, 0.082034, 0.011844, 0.032342, 0.087551, 0.011282, 0.021407, 0.034942, 0.017503, 0.036321]
    # }
    # ca = pd.DataFrame(data)

    # Prepend cell names
    # df['improve_sample_id'] = 'pdo: ' + df['improve_sample_id']

    # Pivot the DataFrame
    # df_pivot = df.pivot(index=drug_col_name, columns=canc_col_name, values=['pred_mean', 'pred_std'])
    df_pivot = df.pivot(index='drug_name', columns=canc_col_name, values=['pred_mean', 'pred_std'])

    # df_pivot = df_pivot.sort_values(('pred_mean', '655913~031-T'))
    random_pdo = df['improve_sample_id'].unique()[0]
    df_pivot = df_pivot.sort_values(('pred_mean', random_pdo))
    
    # Plotting
    fig, ax = plt.subplots(figsize=(14, 8))

    # Get the number of drugs and cells
    # drugs = df[drug_col_name].unique()
    drugs = df['drug_name'].unique()
    cells = df[canc_col_name].unique()
    bar_width = 0.4  # Width of the bars
    index = np.arange(len(drugs))  # The label locations

    # Use a color palette from seaborn
    # colors = sns.color_palette("terrain_r", len(cells))
    # colors = sns.color_palette("husl", 9)
    colors = sns.color_palette("pastel")
    # colors = sns.color_palette("muted", 5)
    # colors = sns.color_palette("blight", 5)
    # colors = sns.color_palette("deep", 10)
    # colors = sns.color_palette()

    # Plot each cell
    for i, (cell, color) in enumerate(zip(cells, colors)):
        pred_mean = df_pivot['pred_mean'][cell].values
        pred_std = df_pivot['pred_std'][cell].values
        if errorbar:
            bars = ax.bar(index + i * bar_width, pred_mean, bar_width, yerr=pred_std,
                          label=cell, color=color, capsize=5, ecolor='black',
                          error_kw=dict(linestyle='--', linewidth=1))
            title = f'Predicted AUC and Standard Deviation by Drug and PDO (trained on {tr_data})'
        else:
            bars = ax.bar(index + i * bar_width, pred_mean, bar_width, label=cell,
                          color=color, capsize=5, ecolor='black')
            title = f'Predicted AUC by Drug and PDO (trained on {tr_data})'

        # Add annotations
        for bar, mean in zip(bars, pred_mean):
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2, yval, f'{mean:.2f}', ha='center', va='bottom')

    # Add labels, title, and legend
    ax.set_xlabel('Drug')
    ax.set_ylabel('Predicted AUC (mean across data splits)')
    ax.set_title(title)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(drugs, rotation=45)
    ax.legend()
    ax.grid()

    ax.set_ylim(0.4, None)

    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(fpath, dpi=300)
    plt.show()


In [ ]:
tr_data = "CTRPv2"
gen_plot(df=ca, fpath=filepath / f'pdmr_trained_{model_name}_{tr_data}.png', tr_data=tr_data, errorbar=False, show=True)

In [ ]:
tr_data = "GDSCv2"
gen_plot(df=ga, fpath=filepath / f'pdmr_trained_{tr_data}.png', tr_data=tr_data, errorbar=False, show=True)